# RAG System for Sport Articles

## Objectives:
1. Develop an RAG system using the semantic retriever from the previous assignment
2. Evaluate the performance of the RAG system

## Pipeline:
`Query → Retriever (FAISS) → Retrieved Passages → Prompt Construction → LLM (Ollama) → Grounded Response`

## Prerequisites:
- Run `assignment-non-rag.ipynb` first to generate embeddings and FAISS index
- Start Ollama: `ollama serve`
- Pull the model: `ollama pull qwen2.5:7b-instruct`

## Step 0: Load Pre-trained Retriever

Load the embeddings, FAISS index, and encoder model from the previous assignment.

In [1]:
import json
import os
import numpy as np
import pandas as pd
import faiss
from sentence_transformers import SentenceTransformer

# Paths
DATA_PATH = "scraping_result/detik_sport_articles_cleaned.json"
EMBEDDINGS_PATH = "embedding/article_embeddings.npy"
INDEX_PATH = "faiss/faiss_index.bin"
MODEL_NAME = "intfloat/multilingual-e5-base"

# Verify files exist
required_files = [DATA_PATH, EMBEDDINGS_PATH, INDEX_PATH]
for file_path in required_files:
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"❌ {file_path} not found. Run assignment-non-rag.ipynb first!")

# Load articles data
with open(DATA_PATH, "r", encoding="utf-8") as f:
    articles = json.load(f)
df = pd.DataFrame(articles)
df = df[df['content'].str.len() > 50].reset_index(drop=True)

# Load embeddings and FAISS index
embeddings = np.load(EMBEDDINGS_PATH)
index = faiss.read_index(INDEX_PATH)

# Load encoder model
print(f"Loading encoder model: {MODEL_NAME}")
encoder_model = SentenceTransformer(MODEL_NAME, device="cpu")

print(f"✅ Loaded {len(df)} articles")
print(f"✅ Loaded embeddings: {embeddings.shape}")
print(f"✅ Loaded FAISS index: {index.ntotal} vectors")
print(f"✅ Encoder model ready")

Loading encoder model: intfloat/multilingual-e5-base
✅ Loaded 396 articles
✅ Loaded embeddings: (396, 768)
✅ Loaded FAISS index: 396 vectors
✅ Encoder model ready
✅ Loaded 396 articles
✅ Loaded embeddings: (396, 768)
✅ Loaded FAISS index: 396 vectors
✅ Encoder model ready


## Step 1: Select Generation Component

### Model Choice: **Qwen2.5:7b-instruct**

**Key reasons:**
1. **Better Indonesian understanding** - Our articles are in Indonesian
2. **Larger context window** - Can include more retrieved passages
3. **Better instruction following** - More likely to answer based on context only

In [2]:
import requests

# Ollama API configuration
OLLAMA_BASE_URL = "http://localhost:11434"
LLM_MODEL = "qwen2.5:7b-instruct"  # or "qwen2.5:7b" if instruct not available

def check_ollama_status():
    """Check if Ollama is running and model is available"""
    try:
        # Check if Ollama is running
        response = requests.get(f"{OLLAMA_BASE_URL}/api/tags")
        if response.status_code == 200:
            models = response.json().get('models', [])
            model_names = [m['name'] for m in models]
            print(f"✅ Ollama is running")
            print(f"📦 Available models: {model_names}")

            # Check if our model is available
            if any(LLM_MODEL in name for name in model_names):
                print(f"✅ Model '{LLM_MODEL}' is available")
                return True
            else:
                print(f"⚠️ Model '{LLM_MODEL}' not found. Please run: ollama pull {LLM_MODEL}")
                return False
        else:
            print("❌ Ollama API returned error")
            return False
    except requests.exceptions.ConnectionError:
        print("❌ Cannot connect to Ollama. Please start Ollama first: ollama serve")
        return False

# Check Ollama status
ollama_ready = check_ollama_status()

✅ Ollama is running
📦 Available models: ['qwen2.5:7b-instruct']
✅ Model 'qwen2.5:7b-instruct' is available


## Step 2: Build RAG Pipeline

Components:
1. **Retriever**: Reuse FAISS index from Part 1
2. **Prompt Constructor**: Combine query + retrieved passages
3. **Generator**: Ollama with Qwen2.5

In [3]:
def retrieve_passages(query: str, top_k: int = 3) -> list[dict]:
    """
    Retrieve relevant passages using FAISS index.
    """
    # E5 requires "query: " prefix
    query_with_prefix = f"query: {query}"

    # Encode query using the encoder model
    query_embedding = encoder_model.encode(
        [query_with_prefix],
        normalize_embeddings=True,
        convert_to_numpy=True
    )

    # Search FAISS
    scores, indices = index.search(query_embedding.astype('float32'), top_k)

    # Get passages with metadata
    passages = []
    for score, idx in zip(scores[0], indices[0]):
        passages.append({
            'title': df.iloc[idx]['title'],
            'content': df.iloc[idx]['content'],
            'date': df.iloc[idx]['date'],
            'score': float(score)
        })

    return passages


def construct_rag_prompt(query: str, passages: list[dict], max_content_len: int = 400) -> str:
    """
    Construct a prompt that combines the query with retrieved passages.
    Instructs the LLM to answer based ONLY on the provided context.

    Args:
        query: User question
        passages: Retrieved passages from FAISS
        max_content_len: Maximum characters per passage content (default 400 for faster inference)
    """
    # Format passages with shorter content
    context_parts = []
    for i, p in enumerate(passages, 1):
        content_preview = p['content'][:max_content_len]
        context_parts.append(f"[{i}] {p['title']}\n{content_preview}")

    context = "\n\n".join(context_parts)

    # Shorter, more focused prompt
    prompt = f"""Jawab pertanyaan berdasarkan konteks berikut. Jika tidak ada informasi, katakan "Tidak tersedia".

KONTEKS:
{context}

PERTANYAAN: {query}

JAWABAN:"""

    return prompt


def construct_non_rag_prompt(query: str) -> str:
    """
    Construct a prompt WITHOUT retrieved passages (for comparison).
    """
    prompt = f"""Jawab pertanyaan berikut dalam Bahasa Indonesia.

PERTANYAAN: {query}"""

    return prompt


print("✅ RAG pipeline functions defined")

✅ RAG pipeline functions defined


In [4]:
import time

def generate_with_ollama(prompt: str, model: str = LLM_MODEL, timeout: int = 180) -> tuple[str, float]:
    """
    Generate response using Ollama API.

    Args:
        prompt: The prompt to send to the LLM
        model: Ollama model name
        timeout: Request timeout in seconds (default 180 = 3 minutes)

    Returns:
        tuple: (response_text, elapsed_time)
    """
    start_time = time.time()

    try:
        response = requests.post(
            f"{OLLAMA_BASE_URL}/api/generate",
            json={
                "model": model,
                "prompt": prompt,
                "stream": False,
                "options": {
                    "temperature": 0.3,
                    "top_p": 0.9,
                    "num_predict": 256
                }
            },
            timeout=timeout
        )

        elapsed = time.time() - start_time

        if response.status_code == 200:
            return response.json()['response'], elapsed
        else:
            return f"Error: {response.status_code} - {response.text}", elapsed
    except requests.exceptions.Timeout:
        elapsed = time.time() - start_time
        return f"Error: Request timed out after {elapsed:.0f}s. Try a smaller model or shorter context.", elapsed
    except Exception as e:
        elapsed = time.time() - start_time
        return f"Error: {str(e)}", elapsed


def rag_query(query: str, top_k: int = 3, verbose: bool = True) -> dict:
    """
    Complete RAG pipeline: Retrieve → Construct Prompt → Generate
    """
    # Step 1: Retrieve relevant passages
    passages = retrieve_passages(query, top_k=top_k)

    if verbose:
        print(f"🔍 Query: {query}")
        print(f"📚 Retrieved {len(passages)} passages:")
        for i, p in enumerate(passages, 1):
            print(f"   {i}. [{p['score']:.4f}] {p['title']}")
        print()

    # Step 2: Construct RAG prompt
    rag_prompt = construct_rag_prompt(query, passages)

    if verbose:
        print(f"📊 Prompt: {len(rag_prompt)} chars (~{len(rag_prompt)//4} tokens)")
        print("🤖 Generating RAG response...")

    # Step 3: Generate response
    rag_response, elapsed = generate_with_ollama(rag_prompt)

    if verbose:
        print(f"   ⏱️ Generated in {elapsed:.1f}s")

    return {
        'query': query,
        'passages': passages,
        'rag_response': rag_response,
        'rag_prompt': rag_prompt,
        'elapsed': elapsed
    }


def non_rag_query(query: str, verbose: bool = True) -> dict:
    """
    Generate response WITHOUT retrieval (for comparison)
    """
    prompt = construct_non_rag_prompt(query)

    if verbose:
        print(f"🔍 Query: {query}")
        print("🤖 Generating non-RAG response (no context)...")

    response, elapsed = generate_with_ollama(prompt)

    if verbose:
        print(f"   ⏱️ Generated in {elapsed:.1f}s")

    return {
        'query': query,
        'non_rag_response': response,
        'elapsed': elapsed
    }


print("✅ Generation functions defined")

✅ Generation functions defined


## Step 3: Evaluation

Test the RAG system with different types of questions:
1. **Factual queries** - Specific facts that should be in the articles
2. **Conceptual understanding** - Questions requiring comprehension
3. **Synthesis questions** - Questions requiring combining information from multiple sources

In [5]:
# Define test questions for different categories
test_questions = {
    "factual": [
        "Siapa pelatih baru Timnas Indonesia?",
        "Berapa gol yang dicetak Ronaldo di Piala Dunia 2026?",
        "Kapan Barcelona membeli pemain baru terakhir kali?"
    ],
    "conceptual": [
        "Mengapa Manchester United mengalami masalah performa musim ini?",
        "Apa alasan Rashford kesulitan di Manchester United?",
        "Bagaimana kondisi klasemen Liga Inggris saat ini?"
    ],
    "synthesis": [
        "Bandingkan performa Ronaldo dan pemain bintang lainnya di kompetisi internasional",
        "Apa dampak pergantian pelatih terhadap performa tim-tim besar Eropa?",
        "Bagaimana situasi transfer pemain di liga-liga top Eropa?"
    ]
}

print("📋 Test Questions Defined:")
for category, questions in test_questions.items():
    print(f"\n{category.upper()}:")
    for i, q in enumerate(questions, 1):
        print(f"   {i}. {q}")

📋 Test Questions Defined:

FACTUAL:
   1. Siapa pelatih baru Timnas Indonesia?
   2. Berapa gol yang dicetak Ronaldo di Piala Dunia 2026?
   3. Kapan Barcelona membeli pemain baru terakhir kali?

CONCEPTUAL:
   1. Mengapa Manchester United mengalami masalah performa musim ini?
   2. Apa alasan Rashford kesulitan di Manchester United?
   3. Bagaimana kondisi klasemen Liga Inggris saat ini?

SYNTHESIS:
   1. Bandingkan performa Ronaldo dan pemain bintang lainnya di kompetisi internasional
   2. Apa dampak pergantian pelatih terhadap performa tim-tim besar Eropa?
   3. Bagaimana situasi transfer pemain di liga-liga top Eropa?


### Test 1: Factual Query

In [6]:
# DEBUG: See what's being sent to the LLM (UNTUK DWBUGGING AJA)
test_query = "Siapa pelatih baru Timnas Indonesia?"
passages = retrieve_passages(test_query, top_k=3)
rag_prompt = construct_rag_prompt(test_query, passages)

In [7]:
print("📊 PROMPT STATISTICS:")
print(f"   - Total characters: {len(rag_prompt)}")
print(f"   - Estimated tokens: ~{len(rag_prompt) // 4} tokens")
print(f"   - Number of passages: {len(passages)}")
print()
print("📝 FULL PROMPT SENT TO LLM:")
print("=" * 80)
print(rag_prompt)
print("=" * 80)

📊 PROMPT STATISTICS:
   - Total characters: 1540
   - Estimated tokens: ~385 tokens
   - Number of passages: 3

📝 FULL PROMPT SENT TO LLM:
Jawab pertanyaan berdasarkan konteks berikut. Jika tidak ada informasi, katakan "Tidak tersedia".

KONTEKS:
[1] Gabung Navbahor, Kapadze Dipastikan Tidak Latih Timnas Indonesia
Jakarta - Timur Kapadze dipastikan tidak akan menjadi pelatih Timnas Indonesia . Dia sudah menerima pinangan klub Navbahor Namangan. "Navbahor hari ini secara resmi memperkenalkan Timur Kapadze sebagai pelatih kepala baru. Kedua pihak telah menandatangani kontrak kerja sama," bunyi pernyataan klub di media sosial, Senin . Navbahor adalah klub asal Uzbekistan yang sudah berdiri sejak 1978. Kapadze 

[2] SEA Games 2025: Timnas Basket Coret 3 Nama
Jakarta - Timnas basket Indonesia akan kembali mencoret tiga nama. Itu sebagai bagian dari persiapan mengikuti rangkaian uji coba di Australia menuju SEA Games 2025. Diketahui, Timnas telah menjalani pemusatan latihan sejak 10 Septembe

In [8]:
# Factual Query Test
factual_query = "Siapa pelatih baru Timnas Indonesia?"

# RAG Response
factual_rag_result = rag_query(factual_query, top_k=3)

# Non-RAG Response
factual_non_rag_result = non_rag_query(factual_query)

🔍 Query: Siapa pelatih baru Timnas Indonesia?
📚 Retrieved 3 passages:
   1. [0.8454] Gabung Navbahor, Kapadze Dipastikan Tidak Latih Timnas Indonesia
   2. [0.8248] SEA Games 2025: Timnas Basket Coret 3 Nama
   3. [0.8186] BTN Panggil 24 Pebasket Ikuti TC SEA Games 2025

📊 Prompt: 1540 chars (~385 tokens)
🤖 Generating RAG response...
   ⏱️ Generated in 48.5s
🔍 Query: Siapa pelatih baru Timnas Indonesia?
🤖 Generating non-RAG response (no context)...
   ⏱️ Generated in 48.5s
🔍 Query: Siapa pelatih baru Timnas Indonesia?
🤖 Generating non-RAG response (no context)...
   ⏱️ Generated in 16.4s
   ⏱️ Generated in 16.4s


In [9]:
print("=" * 80)
print("FACTUAL QUERY TEST")
print("=" * 80)

print("\n📗 RAG RESPONSE:")
print("-" * 40)
print(factual_rag_result['rag_response'])

print("\n" + "=" * 80)

print("\n📕 NON-RAG RESPONSE:")
print("-" * 40)
print(factual_non_rag_result['non_rag_response'])

FACTUAL QUERY TEST

📗 RAG RESPONSE:
----------------------------------------
Pelatih baru Timnas Indonesia adalah Timur Kapadze, yang sekarang menjadi pelatih kepala baru untuk klub Navbahor Namangan di Uzbekistan.


📕 NON-RAG RESPONSE:
----------------------------------------
Maaf, saya tidak memiliki informasi terbaru tentang pelatih tim nasional Indonesia karena data yang saya miliki terbatas dan tidak selalu up-to-date. Untuk mendapatkan informasi terkini mengenai pelatih tim nasional Indonesia, Anda bisa mengunjungi situs resmi PSSI (Persatuan Sepakbola Seluruh Indonesia) atau media berita yang terpercaya.


### Test 2: Conceptual Understanding

In [10]:

# Conceptual Query Test
conceptual_query = "Mengapa Rashford kesulitan di Manchester United?"

# RAG Response
conceptual_rag_result = rag_query(conceptual_query, top_k=3)

# Non-RAG Response
conceptual_non_rag_result = non_rag_query(conceptual_query)

🔍 Query: Mengapa Rashford kesulitan di Manchester United?
📚 Retrieved 3 passages:
   1. [0.8942] Deco: Rashford Menderita di MU
   2. [0.8361] Flick: Rashford Enjoy Banget di Barcelona
   3. [0.8296] Ruben Amorim Sadar Patrick Dorgu Cemas Tiap Kuasai Bola

📊 Prompt: 1525 chars (~381 tokens)
🤖 Generating RAG response...
   ⏱️ Generated in 70.3s
🔍 Query: Mengapa Rashford kesulitan di Manchester United?
🤖 Generating non-RAG response (no context)...
   ⏱️ Generated in 70.3s
🔍 Query: Mengapa Rashford kesulitan di Manchester United?
🤖 Generating non-RAG response (no context)...
   ⏱️ Generated in 46.9s
   ⏱️ Generated in 46.9s


In [11]:
print("=" * 80)
print("CONCEPTUAL UNDERSTANDING TEST")
print("=" * 80)

print("\n📗 RAG RESPONSE:")
print("-" * 40)
print(conceptual_rag_result['rag_response'])

print("\n" + "=" * 80)

print("\n📕 NON-RAG RESPONSE:")
print("-" * 40)
print(conceptual_non_rag_result['non_rag_response'])


CONCEPTUAL UNDERSTANDING TEST

📗 RAG RESPONSE:
----------------------------------------
Menurut konteks yang diberikan, Marcus Rashford kesulitan di Manchester United karena beberapa alasan:

1. Dia dipinjamkan oleh Manchester United ke Barcelona pada musim panas lalu.
2. Setelah tidak masuk rencana pelatih Ruben Amorim di Barcelona, dia kembali dipinjamkan oleh MU dan ini merupakan kali kedua secara beruntun.

Kesulitan Rashford di Manchester United mungkin juga terkait dengan kurangnya waktu bermain yang cukup di bawah naungan Man United akibat pinjaman tersebut.


📕 NON-RAG RESPONSE:
----------------------------------------
Rashford mengalami beberapa masalah yang dapat menyebabkan dia kesulitan di Manchester United, termasuk:

1. Tekanan Emosional: Sebagai salah satu pemain muda berbakat dan menjadi bagian penting dari skuad Manchester United sejak usia dini, Rashford sering menghadapi tekanan besar untuk terus memberikan performa yang luar biasa.

2. Persaingan Ketat: Manchester U

### Test 3: Synthesis Question

In [12]:
# Synthesis Query Test
synthesis_query = "Bagaimana situasi transfer pemain di liga-liga top Eropa?"

# RAG Response
synthesis_rag_result = rag_query(synthesis_query, top_k=5)  # More passages for synthesis

# Non-RAG Response
synthesis_non_rag_result = non_rag_query(synthesis_query)


🔍 Query: Bagaimana situasi transfer pemain di liga-liga top Eropa?
📚 Retrieved 5 passages:
   1. [0.8229] FIFA Izinkan Klub-klub Premier League Telat Lepas Pemainnya ke Piala Afrika
   2. [0.8158] Aturan Substitusi: Dari 3 Pemain hingga 5 Pemain
   3. [0.8110] Nico Paz Balik ke Real Madrid? Fabregas Kasih Komentar Tajam
   4. [0.8103] Szczesny 'Tak Gajian' di Musim Pertamanya di Barca
   5. [0.8091] Courtois: Selalu Ada Pemain yang Kurang Bahagia di Ruang Ganti Madrid

📊 Prompt: 2524 chars (~631 tokens)
🤖 Generating RAG response...
   ⏱️ Generated in 96.7s
🔍 Query: Bagaimana situasi transfer pemain di liga-liga top Eropa?
🤖 Generating non-RAG response (no context)...
   ⏱️ Generated in 96.7s
🔍 Query: Bagaimana situasi transfer pemain di liga-liga top Eropa?
🤖 Generating non-RAG response (no context)...
   ⏱️ Generated in 47.8s
   ⏱️ Generated in 47.8s


In [14]:

print("=" * 80)
print("SYNTHESIS QUESTION TEST")
print("=" * 80)

print("\n📗 RAG RESPONSE:")
print("-" * 40)
print(synthesis_rag_result['rag_response'])

print("\n" + "=" * 80)

print("\n📕 NON-RAG RESPONSE:")
print("-" * 40)
print(synthesis_non_rag_result['non_rag_response'])

SYNTHESIS QUESTION TEST

📗 RAG RESPONSE:
----------------------------------------
Tidak tersedia.

Konteks yang diberikan tidak menyediakan informasi spesifik tentang situasi transfer pemain di liga-liga top Eropa. Konten tersebut membahas mengenai izin FIFA untuk Premier League, aturan substitusi dalam sepakbola, Nico Paz dan Real Madrid, Szczesny pensiun dari Barcelona, serta ketegangan di ruang ganti Real Madrid. Tidak ada informasi yang spesifik tentang transfer pemain secara umum di liga-liga top Eropa.


📕 NON-RAG RESPONSE:
----------------------------------------
Situasi transfer pemain di liga-liga top Eropa sangat dinamis dan penuh dengan kegiatan. Transfer pemain biasanya terjadi pada musim panas (juni-agustus) dan musim dingin (januari) setiap tahunnya, meskipun ada beberapa pengecualian.

1. **Aktivitas Transfensi**: Klub-klub besar di liga-liga top Eropa seperti Premier League Inggris, La Liga Spanyol, Serie A Italia, Bundesliga Jerman, dan Ligue 1 Prancis sering kali mela

## Step 4: Qualitative Evaluation Framework

Since automated evaluation of RAG systems can be unreliable, we use a **human evaluation framework** with clear criteria.

### Evaluation Criteria:

| Criterion | Description | Score Range |
|-----------|-------------|-------------|
| **Groundedness** | Is the answer based on retrieved passages? | 1-5 |
| **Factual Accuracy** | Are the stated facts correct? | 1-5 |
| **Hallucination** | Does it avoid making up information? | 1-5 |
| **Relevance** | Does it answer the question? | 1-5 |
| **Completeness** | Is the answer comprehensive? | 1-5 |

In [ ]:
# Run comprehensive evaluation on all three test types
print("=" * 80)
print("COMPREHENSIVE EVALUATION SUMMARY")
print("=" * 80)

# Collect all results
all_results = {
    "Factual": {
        "query": factual_query,
        "rag": factual_rag_result,
        "non_rag": factual_non_rag_result
    },
    "Conceptual": {
        "query": conceptual_query,
        "rag": conceptual_rag_result,
        "non_rag": conceptual_non_rag_result
    },
    "Synthesis": {
        "query": synthesis_query,
        "rag": synthesis_rag_result,
        "non_rag": synthesis_non_rag_result
    }
}

# Display summary table
print("\n📊 Response Time Comparison:")
print("-" * 60)
print(f"{'Query Type':<15} {'RAG Time':<15} {'Non-RAG Time':<15}")
print("-" * 60)
for qtype, results in all_results.items():
    rag_time = results['rag']['elapsed']
    non_rag_time = results['non_rag']['elapsed']
    print(f"{qtype:<15} {rag_time:.1f}s{'':<10} {non_rag_time:.1f}s")
print("-" * 60)

# Display retrieved passages for each query
print("\n📚 Retrieved Passages Summary:")
for qtype, results in all_results.items():
    print(f"\n{qtype}: {results['query'][:50]}...")
    for i, p in enumerate(results['rag']['passages'], 1):
        print(f"   {i}. [{p['score']:.4f}] {p['title'][:60]}...")

### Manual Evaluation Template

Use the cell below to manually score the responses. Fill in scores 1-5 for each criterion.

In [ ]:
# Manual Evaluation Scores (Fill in after reviewing responses above)
# Score range: 1 (Poor) to 5 (Excellent)

manual_scores = {
    "RAG": {
        "Factual": {
            "groundedness": 0,      # Is answer based on retrieved passages?
            "factual_accuracy": 0,  # Are facts correct?
            "no_hallucination": 0,  # Does it avoid making things up?
            "relevance": 0,         # Does it answer the question?
            "completeness": 0       # Is the answer complete?
        },
        "Conceptual": {
            "groundedness": 0,
            "factual_accuracy": 0,
            "no_hallucination": 0,
            "relevance": 0,
            "completeness": 0
        },
        "Synthesis": {
            "groundedness": 0,
            "factual_accuracy": 0,
            "no_hallucination": 0,
            "relevance": 0,
            "completeness": 0
        }
    },
    "Non-RAG": {
        "Factual": {
            "groundedness": 0,
            "factual_accuracy": 0,
            "no_hallucination": 0,
            "relevance": 0,
            "completeness": 0
        },
        "Conceptual": {
            "groundedness": 0,
            "factual_accuracy": 0,
            "no_hallucination": 0,
            "relevance": 0,
            "completeness": 0
        },
        "Synthesis": {
            "groundedness": 0,
            "factual_accuracy": 0,
            "no_hallucination": 0,
            "relevance": 0,
            "completeness": 0
        }
    }
}

print("📝 Manual Evaluation Template")
print("=" * 60)
print("\nInstructions:")
print("1. Review the RAG and Non-RAG responses from the tests above")
print("2. Fill in scores (1-5) in the manual_scores dictionary")
print("3. Run the next cell to see the summary")
print("\nScoring Guide:")
print("  1 = Poor       - Completely wrong or irrelevant")
print("  2 = Below Avg  - Major issues")
print("  3 = Average    - Acceptable but with issues")
print("  4 = Good       - Mostly correct with minor issues")
print("  5 = Excellent  - Fully correct and complete")

In [ ]:
def calculate_evaluation_summary(scores: dict) -> pd.DataFrame:
    """
    Calculate summary statistics from manual evaluation scores.
    """
    criteria = ["groundedness", "factual_accuracy", "no_hallucination", "relevance", "completeness"]

    summary = []
    for system in ["RAG", "Non-RAG"]:
        system_scores = scores[system]

        # Calculate averages across all queries
        avg_scores = {c: [] for c in criteria}

        for query_scores in system_scores.values():
            for criterion, score in query_scores.items():
                if score > 0:  # Only count filled scores
                    avg_scores[criterion].append(score)

        # Calculate final averages
        final_scores = {}
        for criterion, scores_list in avg_scores.items():
            final_scores[criterion] = sum(scores_list) / len(scores_list) if scores_list else 0

        overall = sum(final_scores.values()) / len(final_scores) if any(final_scores.values()) else 0

        summary.append({
            "System": system,
            "Groundedness": final_scores["groundedness"],
            "Factual Acc.": final_scores["factual_accuracy"],
            "No Halluc.": final_scores["no_hallucination"],
            "Relevance": final_scores["relevance"],
            "Completeness": final_scores["completeness"],
            "Overall": overall
        })

    return pd.DataFrame(summary)


# Calculate and display summary
print("📊 Evaluation Summary")
print("=" * 80)

# Check if scores have been filled
has_scores = any(
    score > 0
    for system in manual_scores.values()
    for query in system.values()
    for score in query.values()
)

if not has_scores:
    print("⚠️ No scores filled yet. Please fill in manual_scores above and re-run this cell.")
else:
    summary_df = calculate_evaluation_summary(manual_scores)
    print(summary_df.to_string(index=False))

    # Calculate winner for each criterion
    print("\n" + "-" * 80)
    print("🏆 Winner by Criterion:")
    for col in summary_df.columns[1:]:
        rag_score = summary_df.loc[summary_df['System'] == 'RAG', col].values[0]
        non_rag_score = summary_df.loc[summary_df['System'] == 'Non-RAG', col].values[0]
        if rag_score > non_rag_score:
            print(f"   {col}: RAG wins ({rag_score:.2f} vs {non_rag_score:.2f})")
        elif non_rag_score > rag_score:
            print(f"   {col}: Non-RAG wins ({non_rag_score:.2f} vs {rag_score:.2f})")
        else:
            print(f"   {col}: Tie ({rag_score:.2f})")

## Step 5: RAG System Analysis & Conclusions

### How to Evaluate RAG Quality

Since automated metrics can be unreliable for RAG evaluation, here are the recommended approaches:

#### 1. **Human Evaluation (Most Reliable)**
- Have evaluators score responses on the criteria above
- Use multiple evaluators and calculate inter-rater agreement
- Best for final quality assessment

#### 2. **Automated Heuristics (Supplementary)**
- Check if response contains keywords from retrieved passages
- Measure response length and structure
- Detect obvious hallucination patterns (e.g., citing non-existent sources)

#### 3. **A/B Testing (Production)**
- Compare user satisfaction between RAG and non-RAG
- Measure engagement metrics (clicks, time spent)

### Expected RAG vs Non-RAG Differences:

| Aspect | RAG | Non-RAG |
|--------|-----|---------|
| **Factual Accuracy** | ✅ Higher (grounded in documents) | ⚠️ May hallucinate |
| **Up-to-date Info** | ✅ Uses recent articles | ❌ Knowledge cutoff |
| **Source Attribution** | ✅ Can cite sources | ❌ No sources |
| **Domain Specificity** | ✅ Specific to your corpus | ⚠️ General knowledge |
| **Handling Unknown** | ✅ Says "not in documents" | ⚠️ May make up answers |

In [ ]:
# Simple automated heuristic evaluation
def automated_heuristics(query: str, rag_response: str, passages: list[dict]) -> dict:
    """
    Simple automated checks (NOT a replacement for human evaluation)
    """
    # Check if response mentions retrieved content
    passage_keywords = set()
    for p in passages:
        # Extract key terms from titles
        words = p['title'].lower().split()
        passage_keywords.update([w for w in words if len(w) > 3])

    response_lower = rag_response.lower()
    keyword_mentions = sum(1 for kw in passage_keywords if kw in response_lower)

    # Check for "information not available" patterns (good sign of not hallucinating)
    uncertainty_phrases = [
        "tidak tersedia", "tidak ditemukan", "tidak ada informasi",
        "berdasarkan artikel", "menurut artikel", "dalam konteks"
    ]
    shows_uncertainty = any(phrase in response_lower for phrase in uncertainty_phrases)
    cites_source = any(phrase in response_lower for phrase in ["menurut artikel", "berdasarkan artikel", "dalam artikel"])

    return {
        "keyword_overlap": keyword_mentions,
        "total_passage_keywords": len(passage_keywords),
        "shows_appropriate_uncertainty": shows_uncertainty,
        "cites_sources": cites_source,
        "response_length": len(rag_response.split())
    }


# Test the heuristics
test_query = "Siapa pelatih Timnas Indonesia?"
test_result = rag_query(test_query, top_k=3, verbose=False)

heuristics = automated_heuristics(
    test_query,
    test_result['rag_response'],
    test_result['passages']
)

print("🔍 Automated Heuristics (Supplementary Only)")
print("=" * 50)
print(f"Query: {test_query}")
print(f"\nHeuristic Results:")
for key, value in heuristics.items():
    print(f"  - {key}: {value}")

## Summary: RAG System Architecture

```
┌─────────────────────────────────────────────────────────────────┐
│                     RAG SYSTEM PIPELINE                        │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   User Query                                                    │
│       │                                                         │
│       ▼                                                         │
│   ┌─────────────────────────────────────┐                      │
│   │  RETRIEVER (from assignment-non-rag)│                      │
│   │  - Multilingual E5 Encoder          │                      │
│   │  - FAISS Vector Index               │                      │
│   │  - Top-K Similar Documents          │                      │
│   └─────────────────────────────────────┘                      │
│       │                                                         │
│       ▼                                                         │
│   ┌─────────────────────────────────────┐                      │
│   │  PROMPT CONSTRUCTOR                 │                      │
│   │  - Combine Query + Passages         │                      │
│   │  - Add Grounding Instructions       │                      │
│   └─────────────────────────────────────┘                      │
│       │                                                         │
│       ▼                                                         │
│   ┌─────────────────────────────────────┐                      │
│   │  GENERATOR (Ollama)                 │                      │
│   │  - Qwen2.5-7B-Instruct              │                      │
│   │  - Grounded Response Generation     │                      │
│   └─────────────────────────────────────┘                      │
│       │                                                         │
│       ▼                                                         │
│   Grounded Response                                             │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### Key Takeaways:
1. **RAG improves factual accuracy** by grounding responses in retrieved documents
2. **Human evaluation is essential** - automated metrics are supplementary
3. **RAG reduces hallucination** when properly instructed to use only context
4. **Trade-off**: RAG responses are limited to corpus knowledge

### Files Generated:
- `assignment-non-rag.ipynb`: Semantic search with FAISS (prerequisite)
- `assignment-rag.ipynb`: RAG system with Ollama (this notebook)